# 🏎️ Projet F1 Data Science – Acquisition 2025
Ce notebook automatise **l’acquisition, la préparation et l’audit qualité** des données F1 pour la saison 2025 à partir de l’API FastF1 et de sources ouvertes (OpenFlights pour la logistique).
> ⚠️ **Remarque** : Les données d’un Grand Prix ne sont pas toujours disponibles immédiatement après la course (dépendance FIA). Le pipeline gère ce cas et n’intègre que les GP dont les données sont effectivement accessibles au moment de l’exécution. Un log de disponibilité est affiché pour chaque GP traité.


## 1️⃣ Imports & Setup
Préparation de l’environnement, import des librairies et configuration des dossiers de cache et de données.


In [19]:
# Import des bibliothèques nécessaires 
from pathlib import Path
import pandas as pd
import fastf1
from datetime import date
from math import radians, sin, cos, sqrt, atan2

In [20]:
# Activation du cache (création du dossier si besoin)
cache_dir = Path('../.cache_f1')
cache_dir.mkdir(exist_ok=True)
fastf1.Cache.enable_cache(cache_dir)

data_dir = Path('../data')
data_dir.mkdir(exist_ok=True)

# Récupération du calendrier de la saison 2025
YEAR = 2025
schedule = fastf1.get_event_schedule(YEAR, include_testing=False)
today = pd.Timestamp(date.today())
completed = schedule[schedule['EventDate'] <= today]

# initialisation des listes pour collecter les données

results_list = []
quali_list = []
weather_list = []
pits_list = []
standings_driver = []
standings_team = []
flight_legs = []

last_airport = None
last_loc = None

## 2️⃣ Logistique & Mapping aéroports ✈️
Téléchargement de la base OpenFlights et mapping circuits F1 → principaux aéroports (IATA).

In [21]:
# Téléchargement de la table aéroports OpenFlights + mapping F1 circuits → IATA

# Téléchargement et prépa du CSV OpenFlights
airports_df = pd.read_csv(
    'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',
    header=None,
    names=[
        'AirportID', 'Name', 'City', 'Country', 'IATA', 'ICAO',
        'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 'Tz',
        'Type', 'Source'
    ],
    dtype={'IATA': str}
)
airports_df = airports_df[airports_df['IATA'].notna() & (airports_df['IATA'].str.len() == 3)]

In [22]:
# Table F1 circuits → principaux aéroports (complète pour le calendrier 2025) : codes aéroport IATA
# Note : les aéroports sont choisis en fonction de leur utilisation par les équipes F1 pour la logistique
# Sources :
# - https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
# - Connaissances générales F1/logistique (sites officiels F1, Wikipedia, forums spécialisés, presse F1)
CIRCUIT_IATA = {
    'Melbourne': 'MEL',
    'Shanghai': 'PVG',
    'Suzuka': 'NGO',
    'Sakhir': 'BAH',
    'Jeddah': 'JED',
    'Miami': 'MIA',
    'Imola': 'BLQ',
    'Monaco': 'NCE',
    'Barcelona': 'BCN',
    'Montréal': 'YUL',
    'Spielberg': 'VIE',
    'Silverstone': 'LHR',
    'Spa-Francorchamps': 'LGG',
    'Budapest': 'BUD',
    'Zandvoort': 'AMS',
    'Monza': 'MXP',
    'Baku': 'GYD',
    'Marina Bay': 'SIN',
    'Austin': 'AUS',
    'Mexico City': 'MEX',
    'São Paulo': 'GRU',
    'Las Vegas': 'LAS',
    'Lusail': 'DOH',
    'Yas Island': 'AUH',
}

## 3️⃣ Fonctions utilitaires
Pour la logistique (distance, coordonnées) et extraction des vrais pitstops.

In [23]:
# Fonction pour obtenir les coordonnées d'un IATA (aéroport)
def get_coords_for_iata(iata):
    row = airports_df[airports_df['IATA'] == iata]
    if not row.empty:
        return float(row['Latitude'].iloc[0]), float(row['Longitude'].iloc[0])
    else:
        return None, None

# Fonction Haversine (distance km entre deux lat/lon)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

#
def extract_real_pitstops(laps_df):
    """
    Extrait les vrais arrêts aux stands détectés par changement de Stint ET/OU changement de Compound.
    Ajoute des colonnes d'intérêt pour enrichir l'analyse stratégique.
    """
    pitstops = []
    for driver, d in laps_df.groupby('Driver'):
        d = d.sort_values('LapNumber')
        prev_row = None
        for idx, row in d.iterrows():
            # Sauter si info stint ou compound absente
            if prev_row is not None and pd.notna(row['Stint']) and pd.notna(prev_row['Stint']):
                stint_change = row['Stint'] != prev_row['Stint']
                compound_change = row['Compound'] != prev_row['Compound']
                if stint_change and compound_change:
                    pitstops.append({
                        'Driver': driver,
                        'DriverNumber': row['DriverNumber'],
                        'TeamName': row['Team'],
                        'LapIn': prev_row['LapNumber'],
                        'LapOut': row['LapNumber'],
                        'PitInTime': prev_row['PitInTime'],
                        'PitOutTime': row['PitOutTime'],
                        'CompoundIn': prev_row['Compound'],
                        'CompoundOut': row['Compound'],
                        'TyreLifeIn': prev_row['TyreLife'],
                        'TyreLifeOut': row['TyreLife'],
                        'PositionIn': prev_row['Position'],
                        'PositionOut': row['Position'],
                        'event': row['event'],
                        'round': row['round']
                    })
            prev_row = row
    return pd.DataFrame(pitstops)


## 4️⃣ Récupération des données F1 & logistique
Pour chaque GP disponible, on télécharge les résultats, qualifs, météo, pitstops, standings, et on calcule la logistique CO₂.

In [24]:
# Aperçu des gp 2025
for _, ev in schedule.iterrows():
    rd = int(ev['RoundNumber'])
    loc = ev['Location']
    event_date = ev['EventDate']
    available = "✅" if event_date <= today else "⏳"
    print(f"{available} Round {rd:02d} – {loc} (date: {event_date.date()})")

✅ Round 01 – Melbourne (date: 2025-03-16)
✅ Round 02 – Shanghai (date: 2025-03-23)
✅ Round 03 – Suzuka (date: 2025-04-06)
✅ Round 04 – Sakhir (date: 2025-04-13)
✅ Round 05 – Jeddah (date: 2025-04-20)
✅ Round 06 – Miami (date: 2025-05-04)
✅ Round 07 – Imola (date: 2025-05-18)
✅ Round 08 – Monaco (date: 2025-05-25)
✅ Round 09 – Barcelona (date: 2025-06-01)
✅ Round 10 – Montréal (date: 2025-06-15)
⏳ Round 11 – Spielberg (date: 2025-06-29)
⏳ Round 12 – Silverstone (date: 2025-07-06)
⏳ Round 13 – Spa-Francorchamps (date: 2025-07-27)
⏳ Round 14 – Budapest (date: 2025-08-03)
⏳ Round 15 – Zandvoort (date: 2025-08-31)
⏳ Round 16 – Monza (date: 2025-09-07)
⏳ Round 17 – Baku (date: 2025-09-21)
⏳ Round 18 – Marina Bay (date: 2025-10-05)
⏳ Round 19 – Austin (date: 2025-10-19)
⏳ Round 20 – Mexico City (date: 2025-10-26)
⏳ Round 21 – São Paulo (date: 2025-11-09)
⏳ Round 22 – Las Vegas (date: 2025-11-22)
⏳ Round 23 – Lusail (date: 2025-11-30)
⏳ Round 24 – Yas Island (date: 2025-12-07)


In [ ]:
for _, ev in completed.iterrows():
    rd = int(ev['RoundNumber'])
    loc = ev['Location']
    print(f"\n▶ Round {rd:02d} – {loc}")

    try:
        # Sessions F1
        race = fastf1.get_session(YEAR, rd, 'R')
        race.load()
        results_list.append(race.results.assign(round=rd, event=loc))
        laps = race.laps.assign(round=rd, event=loc)
        pits_real = extract_real_pitstops(laps)
        pits_list.append(pits_real)
        weather_list.append(race.weather_data.assign(round=rd, event=loc))
        print(f"Colonnes résultats: {race.results.columns.tolist()}")  #  Debug: afficher les colonnes des résultats
        pilot_vars = ['DriverNumber', 'FullName', 'HeadshotUrl', 'TeamName', 'Position', 'GridPosition', 'Points', 'Status']
        standings_driver.append(race.results[pilot_vars].assign(round=rd, event=loc))
        team_vars = ['TeamName', 'TeamColor']
        teams = race.results[team_vars].drop_duplicates()
        team_points = race.results.groupby('TeamName')['Points'].sum().reset_index()
        teams = teams.merge(team_points, on='TeamName')
        standings_team.append(teams.assign(round=rd, event=loc))

    except Exception as e:
        print(f"  ❌ Erreur lors du chargement des données course pour {loc} : {e}")
        continue
    # Qualifications
    try:
        qual = fastf1.get_session(YEAR, rd, 'Q')
        qual.load()
        quali_list.append(qual.results.assign(round=rd, event=loc))
    except Exception as e:
        print(f"  (pas de session Q trouvée) → {e}")

    # Logistique CO₂
    airport = CIRCUIT_IATA.get(loc, None)
    if last_airport and airport:
        lat1, lon1 = get_coords_for_iata(last_airport)
        lat2, lon2 = get_coords_for_iata(airport)
        if None not in (lat1, lon1, lat2, lon2):
            dist_km = haversine(lat1, lon1, lat2, lon2)
            flight_legs.append({
                'from': last_airport, 'to': airport, 'distance_km': dist_km,
                'event_from': last_loc, 'event_to': loc
            })
    last_airport = airport
    last_loc = loc

## 5️⃣ Construction des DataFrames finaux
On concatène toutes les listes pour obtenir les datasets finaux.

In [26]:

df_results    = pd.concat(results_list, ignore_index=True)
df_quali      = pd.concat(quali_list,   ignore_index=True)
df_pits       = pd.concat(pits_list,    ignore_index=True)
df_weather    = pd.concat(weather_list, ignore_index=True)
df_drv_stand  = pd.concat(standings_driver, ignore_index=True)
df_team_stand = pd.concat(standings_team,   ignore_index=True)
df_flights    = pd.DataFrame(flight_legs)


## 6️⃣ Audit qualité rapide 🕵️‍♂️
On vérifie les valeurs manquantes principales pour chaque dataset.

In [27]:
def missing_report(df):
    miss = df.isna().sum()
    pct = 100 * miss / len(df)
    return pd.DataFrame({'missing_count': miss, 'missing_pct': pct.round(2)}).sort_values('missing_pct', ascending=False).head(10)

print("---- Résultats course ----\n")
display(missing_report(df_results))
print("---- Pitstops ----\n")
display(missing_report(df_pits))
print("---- Weather ----\n")
display(missing_report(df_weather))
print("---- Flight legs ----\n")
display(missing_report(df_flights))
print("---- Driver standings ----\n")
display(missing_report(df_drv_stand))
print("---- Team standings ----\n")
display(missing_report(df_team_stand))
print("---- Qualifying ----\n")
display(missing_report(df_quali))

---- Résultats course ----



,missing_count,missing_pct
Q1,199,100.00
Q3,199,100.00
Q2,199,100.00
Time,27,13.57
DriverNumber,0,0.00
BroadcastName,0,0.00
Abbreviation,0,0.00
TeamId,0,0.00
TeamColor,0,0.00
TeamName,0,0.00


---- Pitstops ----



,missing_count,missing_pct
PitOutTime,12,4.72
PitInTime,12,4.72
Driver,0,0.00
TeamName,0,0.00
DriverNumber,0,0.00
LapOut,0,0.00
LapIn,0,0.00
CompoundIn,0,0.00
CompoundOut,0,0.00
TyreLifeIn,0,0.00


---- Weather ----



...

,missing_count,missing_pct
DriverNumber,0,0.0
FullName,0,0.0
HeadshotUrl,0,0.0
TeamName,0,0.0
Position,0,0.0
GridPosition,0,0.0
Points,0,0.0
Status,0,0.0
round,0,0.0
event,0,0.0


---- Team standings ----



,missing_count,missing_pct
TeamName,0,0.0
TeamColor,0,0.0
Points,0,0.0
round,0,0.0
event,0,0.0


---- Qualifying ----



,missing_count,missing_pct
Points,200,100.0
Time,200,100.0
GridPosition,200,100.0
Q3,101,50.5
Q2,55,27.5
Q1,2,1.0
DriverNumber,0,0.0
TeamId,0,0.0
TeamColor,0,0.0
TeamName,0,0.0


## 7️⃣ Audit valeurs manquantes, Nettoyage & Préparation des données
On ajuste les colonnes inutiles, on documente les NaN, et on prépare les datasets pour la suite du pipeline.

L'objectif est de garantir une des **données propre et fiable** avant de lancer l'analyse exploratoire (EDA).

### ---- Résultats course ----

In [28]:
df_results[df_results['Time'].isna()]

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,round,event
14,30,L LAWSON,LAW,lawson,Red Bull Racing,3671C6,red_bull,Liam,Lawson,Liam Lawson,...,R,18.0,NaT,NaT,NaT,NaT,Retired,0.0,1,Melbourne
15,5,G BORTOLETO,BOR,bortoleto,Kick Sauber,52E252,sauber,Gabriel,Bortoleto,Gabriel Bortoleto,...,R,15.0,NaT,NaT,NaT,NaT,Retired,0.0,1,Melbourne
16,14,F ALONSO,ALO,alonso,Aston Martin,229971,aston_martin,Fernando,Alonso,Fernando Alonso,...,R,12.0,NaT,NaT,NaT,NaT,Retired,0.0,1,Melbourne
17,55,C SAINZ,SAI,sainz,Williams,64C4FF,williams,Carlos,Sainz,Carlos Sainz,...,R,10.0,NaT,NaT,NaT,NaT,Retired,0.0,1,Melbourne
18,7,J DOOHAN,DOO,doohan,Alpine,0093CC,alpine,Jack,Doohan,Jack Doohan,...,R,14.0,NaT,NaT,NaT,NaT,Retired,0.0,1,Melbourne
19,6,I HADJAR,HAD,hadjar,Racing Bulls,6692FF,rb,Isack,Hadjar,Isack Hadjar,...,R,11.0,NaT,NaT,NaT,NaT,Retired,0.0,1,Melbourne
36,14,F ALONSO,ALO,alonso,Aston Martin,229971,aston_martin,Fernando,Alonso,Fernando Alonso,...,R,13.0,NaT,NaT,NaT,NaT,Retired,0.0,2,Shanghai
37,16,C LECLERC,LEC,leclerc,Ferrari,E80020,ferrari,Charles,Leclerc,Charles Leclerc,...,D,6.0,NaT,NaT,NaT,NaT,Disqualified,0.0,2,Shanghai
38,44,L HAMILTON,HAM,hamilton,Ferrari,E80020,ferrari,Lewis,Hamilton,Lewis Hamilton,...,D,5.0,NaT,NaT,NaT,NaT,Disqualified,0.0,2,Shanghai
39,10,P GASLY,GAS,gasly,Alpine,0093CC,alpine,Pierre,Gasly,Pierre Gasly,...,D,16.0,NaT,NaT,NaT,NaT,Disqualified,0.0,2,Shanghai


Les colonnes Q1/Q2/Q3 sont entièrement manquantes dans les résultats de course car elles ne sont renseignées **que lors des qualifications**. Elles seront ignorées ici.  
Les valeurs manquantes dans la colonne “Time” correspondent uniquement à des pilotes ayant abandonné ou ayant été disqualifiés (cf. colonne “Status”).  
**Aucune action de nettoyage n’est nécessaire sur les abandons : ces lignes sont légitimes et doivent être conservées pour l’analyse des abandons/disqualifications.**

In [29]:
 # Suppression des colonnes Q1/Q2/Q3 inutiles ici
df_results = df_results.drop(columns=['Q1', 'Q2', 'Q3'], errors='ignore')

### ---- Pitstops  ----

In [30]:
df_pits[df_pits[['PitInTime', 'PitOutTime']].isna().any(axis=1)]

,Driver,DriverNumber,TeamName,LapIn,LapOut,PitInTime,PitOutTime,CompoundIn,CompoundOut,TyreLifeIn,TyreLifeOut,PositionIn,PositionOut,event,round
126,ALB,23,Williams,32.0,33.0,NaT,NaT,MEDIUM,HARD,8.0,1.0,5.0,5.0,Miami,6
127,ALO,14,Aston Martin,32.0,33.0,NaT,NaT,HARD,MEDIUM,10.0,2.0,15.0,15.0,Miami,6
128,ANT,12,Mercedes,32.0,33.0,NaT,NaT,MEDIUM,HARD,8.0,1.0,6.0,6.0,Miami,6
130,HAM,44,Ferrari,32.0,33.0,NaT,NaT,HARD,MEDIUM,9.0,1.0,9.0,9.0,Miami,6
132,LAW,30,Racing Bulls,31.0,32.0,NaT,NaT,HARD,MEDIUM,8.0,1.0,17.0,17.0,Miami,6
133,LEC,16,Ferrari,32.0,33.0,NaT,NaT,MEDIUM,HARD,8.0,1.0,8.0,8.0,Miami,6
134,NOR,4,McLaren,32.0,33.0,NaT,NaT,MEDIUM,HARD,8.0,1.0,2.0,2.0,Miami,6
135,PIA,81,McLaren,32.0,33.0,NaT,NaT,MEDIUM,HARD,8.0,1.0,1.0,1.0,Miami,6
136,RUS,63,Mercedes,32.0,33.0,NaT,NaT,HARD,MEDIUM,8.0,1.0,3.0,3.0,Miami,6
137,SAI,55,Williams,32.0,33.0,NaT,NaT,MEDIUM,HARD,14.0,1.0,7.0,7.0,Miami,6


Certains arrêts sous Virtual Safety Car (VSC) ou Safety Car peuvent manquer de timestamps précis (`PitInTime`, `PitOutTime`), alors que le changement de pneus a bien eu lieu.  
Cela reflète une limitation du système de chronométrage FIA/FastF1 en condition neutralisée.  
Aucune correction n’est appliquée : les analyses “fréquence/type d’arrêt” restent valides, seule l’étude du delta pit est à filtrer sur valeurs valides.


### ---- Qualifs  ----

In [31]:
df_quali

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,round,event
0,4,L NORRIS,NOR,norris,McLaren,FF8000,mclaren,Lando,Norris,Lando Norris,...,,NaN,0 days 00:01:15.912000,0 days 00:01:15.415000,0 days 00:01:15.096000,NaT,,NaN,1,Melbourne
1,81,O PIASTRI,PIA,piastri,McLaren,FF8000,mclaren,Oscar,Piastri,Oscar Piastri,...,,NaN,0 days 00:01:16.062000,0 days 00:01:15.468000,0 days 00:01:15.180000,NaT,,NaN,1,Melbourne
2,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,...,,NaN,0 days 00:01:16.018000,0 days 00:01:15.565000,0 days 00:01:15.481000,NaT,,NaN,1,Melbourne
3,63,G RUSSELL,RUS,russell,Mercedes,27F4D2,mercedes,George,Russell,George Russell,...,,NaN,0 days 00:01:15.971000,0 days 00:01:15.798000,0 days 00:01:15.546000,NaT,,NaN,1,Melbourne
4,22,Y TSUNODA,TSU,tsunoda,Racing Bulls,6692FF,rb,Yuki,Tsunoda,Yuki Tsunoda,...,,NaN,0 days 00:01:16.225000,0 days 00:01:16.009000,0 days 00:01:15.670000,NaT,,NaN,1,Melbourne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,5,G BORTOLETO,BOR,bortoleto,Kick Sauber,01C00E,sauber,Gabriel,Bortoleto,Gabriel Bortoleto,...,,NaN,0 days 00:01:12.385000,NaT,NaT,NaT,,NaN,10,Montréal
196,55,C SAINZ,SAI,sainz,Williams,1868DB,williams,Carlos,Sainz,Carlos Sainz,...,,NaN,0 days 00:01:12.398000,NaT,NaT,NaT,,NaN,10,Montréal
197,18,L STROLL,STR,stroll,Aston Martin,229971,aston_martin,Lance,Stroll,Lance Stroll,...,,NaN,0 days 00:01:12.517000,NaT,NaT,NaT,,NaN,10,Montréal
198,30,L LAWSON,LAW,lawson,Racing Bulls,6C98FF,rb,Liam,Lawson,Liam Lawson,...,,NaN,0 days 00:01:12.525000,NaT,NaT,NaT,,NaN,10,Montréal


- Les colonnes Points, Time et GridPosition sont toujours manquantes en qualifs (renseignées uniquement après la course).  
  elles seront donc supprimées pour plus de clarté.
- Les NaN dans Q2/Q3 reflètent le format à élimination des qualifications F1 : seules les meilleurs chronos de chaque phase sont renseignés.
- On garde toutes les lignes et NaN pour respecter la logique sportive.


In [32]:
# Suppression des colonnes non pertinentes en qualifs
df_quali = df_quali.drop(columns=['Points', 'Time', 'GridPosition'], errors='ignore')

### ---- Vols et émissions carbone  ----

#### Hypothèses pour le calcul de l’empreinte carbone logistique

- **Masse totale transportée** : 1 400 tonnes par Grand Prix (estimation officielle issue des communiqués DHL, principal partenaire logistique de la F1).
- **Facteur d’émission** : 0,587 kg CO₂ par tonne-kilomètre (t·km), d’après le référentiel officiel UK BEIS 2024 pour le fret aérien longue distance ([source](https://www.gov.uk/government/publications/greenhouse-gas-reporting-conversion-factors-2024)).
- **Calcul appliqué** :  
  > Emissions CO₂ (kg) = distance (km) × masse transportée (t) × facteur d’émission (kg CO₂ / t·km)

**Remarque** : Ces hypothèses reflètent les standards de calcul utilisés par la FIA et l’industrie logistique, garantissant ainsi la pertinence de l’ordre de grandeur de l’analyse.


### 🧮 Calcul CO₂ logistique
On applique les hypothèses officielles FIA/DHL pour estimer l’empreinte carbone des trajets logistiques.

In [33]:
MASS_TONNES = 1400
KG_CO2_PER_TKM = 0.587

df_flights['CO2_kg'] = df_flights['distance_km'] * MASS_TONNES * KG_CO2_PER_TKM
df_flights['CO2_tonnes'] = (df_flights['CO2_kg'] / 1000).round(2)
df_flights['CO2_kg'] = df_flights['CO2_kg'].round(2)
df_flights

,from,to,distance_km,event_from,event_to,CO2_kg,CO2_tonnes
0,MEL,PVG,8017.369472,Melbourne,Shanghai,6588674.23,6588.67
1,PVG,NGO,1456.868760,Shanghai,Suzuka,1197254.75,1197.25
2,NGO,BAH,8052.263175,Suzuka,Sakhir,6617349.88,6617.35
3,BAH,JED,1272.192236,Sakhir,Jeddah,1045487.58,1045.49
4,JED,MIA,11621.236283,Jeddah,Miami,9550331.98,9550.33
5,MIA,BLQ,8149.753826,Miami,Imola,6697467.69,6697.47
6,BLQ,NCE,339.502228,Imola,Monaco,279002.93,279.00
7,NCE,BCN,496.296520,Monaco,Barcelona,407856.48,407.86
8,BCN,YUL,5911.345375,Barcelona,Montréal,4857943.63,4857.94


Une inspection approfondie des résultats a révélé que le pilote Kimi Antonelli apparait sous plusieurs variantes de nom (ex : "Andrea Kimi Antonelli" et "Kimi Antonelli").
Nous allons appliquer une normalisation de ce dernier.


In [34]:
# Pour df_results
mask_results = df_results['DriverId'] == 'antonelli'
df_results.loc[mask_results, 'FirstName'] = 'Kimi'
df_results.loc[mask_results, 'FullName'] = 'Kimi Antonelli'

# Pour df_quali
mask_quali = df_quali['DriverId'] == 'antonelli'
df_quali.loc[mask_quali, 'FirstName'] = 'Kimi'
df_quali.loc[mask_quali, 'FullName'] = 'Kimi Antonelli'

# Pour df_drv_stand
mask_drv = df_drv_stand['FullName'] == 'Andrea Kimi Antonelli'
df_drv_stand.loc[mask_drv, 'FullName'] = 'Kimi Antonelli'

## 8️⃣ Export final 🚀
On sauvegarde tous les datasets au format Parquet pour la suite du pipeline (EDA, ML, dashboard).

In [35]:
# Sauvegarde des DataFrames en fichiers Parquet avec horodatage
#now = datetime.now()
#date_str = now.strftime("%d_%m_%Y_%H_%M")

df_results.to_parquet(data_dir / f'results_2025.parquet')
df_quali.to_parquet(data_dir / f'qualifying_2025.parquet')
df_pits.to_parquet(data_dir / f'pitstops_2025.parquet')
df_weather.to_parquet(data_dir / f'weather_2025.parquet')
df_drv_stand.to_parquet(data_dir / f'driver_standings_2025.parquet')
df_team_stand.to_parquet(data_dir / f'team_standings_2025.parquet')
df_flights.to_parquet(data_dir / f'flightlegs_2025.parquet')